In [11]:
import requests
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import chromedriver_binary
import csv
import time

In [ ]:
'This is not fine'

In [14]:
class Dataset():

    def __init__(self):
        self.URL = 'https://opendata.paris.fr/api/records/1.0/search/?'
        self.dataset = None

    def research_dataset(self, rows=2):
        response = requests.get(self.URL,
                                params={
                                    'dataset': 'terrasses-autorisations',
                                    'rows': rows,
                                    'facet': 'arrondissement',
                                    'facet': 'typologie',
                                    'refine.typologie': 'TERRASSE OUVERTE'
                                }).json()

        dataset = pd.json_normalize(response['records'])
        dataset.columns = dataset.columns.map(
            lambda x: x.replace('fields.', ''))

        return dataset

    def searching(self,siret):
        base_url = 'http://sirene.fr/sirene/public/recherche'
        '''getting the base adress'''
        options = Options().add_argument("--headless")
        options = webdriver.ChromeOptions().add_experimental_option(
            "detach", True)
        driver = webdriver.Chrome(options=options)
        driver.get(base_url)
        '''research query based on list of recipes'''
        search_input = driver.find_element(By.ID, 'sirenSiretQuery')
        search_input.send_keys(siret)
        search_input.submit()
        '''wait time in order to access recipes'''
        wait = WebDriverWait(driver, 5)

        if driver.find_element(
                By.XPATH, "//span[@class='message']"
        ).text.strip(
        ) == "Il n'y a pas de résultat pour votre recherche. Veuillez vérifier les numéros saisis ou modifiez votre recherche.":
            return np.nan

        wait.until(
            ec.visibility_of_element_located(
                (By.XPATH, "//div[@class='alert alert-success null']")))

        enseigne = driver.find_elements(
            By.XPATH, "//div[@class='result-left']/div/p")[0].text.replace(
                'Enseigne :', '').strip()
        denom_usuelle = driver.find_elements(
            By.XPATH, "//div[@class='result-left']/div/p")[1].text.replace(
                'Dénomination usuelle :', '').strip()
        denom = driver.find_elements(
            By.XPATH, "//div[@class='result-right']/p")[2].text.replace(
                'Dénomination :', '').strip()

        driver.quit()

        if enseigne != '':
            print(enseigne)
            return enseigne
        elif denom_usuelle != '':
            print(denom_usuelle)
            return denom_usuelle
        else:
            print(denom)
            return denom

    def complete_dataset(self, rows=2):
        data = self.research_dataset(rows=rows)
        data['enseigne'] = data['siret'].map(self.searching)
        return data

In [15]:
data=Dataset().complete_dataset(rows=15)

LE COMPTOIR
CAFE MERLIN
CAFE GAITE
SOCIAL SQUARE
CASABLANCA
Civilité : MONSIEUR
LA PETITE PERIGOURDINE
CHEZ MADEMOISELLE
CAFE MILOU
PREGO
Civilité : MONSIEUR
L'EPHEMERE


In [10]:
data

,datasetid,recordid,record_timestamp,largeur,typologie,geo_point_2d,lien_affichette,geo_shape.coordinates,geo_shape.type,nom_societe,siret,arrondissement,adresse,periode_installation,longueur,geometry.type,geometry.coordinates,enseigne
0,terrasses-autorisations,37cb18e5bbe495f413efe656cd09535db0df48aa,2022-10-12T05:00:08.974Z,1.00,TERRASSE OUVERTE,"[48.84983668337481, 2.393878879970905]",http://affichette-commerce.paris.fr/2015/Q_44/...,"[2.393878879970905, 48.84983668337481]",Point,SARL LE COMPTOIR,50294703900017,75011,267 BOULEVARD VOLTAIRE,du 01/03 au 01/09,2.84,Point,"[2.393878879970905, 48.84983668337481]",LE COMPTOIR
1,terrasses-autorisations,1a37dd6af382e4b0ddbea93dcd9640397c3fa587,2022-10-12T05:00:08.974Z,0.60,TERRASSE OUVERTE,"[48.859486730815064, 2.3861375748994296]",http://affichette-commerce.paris.fr/2015/Q_43/...,"[2.3861375748994296, 48.859486730815064]",Point,SARL LES RIBATELS,82840204000012,75011,2 RUE MERLIN,Toute l'année,10.80,Point,"[2.3861375748994296, 48.859486730815064]",CAFE MERLIN
2,terrasses-autorisations,63aedf782cbd6c77f9dfe0f4b00f7aa336b3f583,2022-10-12T05:00:08.974Z,6.50,TERRASSE OUVERTE,"[48.8289332873901, 2.3567996474703707]",http://affichette-commerce.paris.fr/2015/Q_51/...,"[2.3567996474703707, 48.8289332873901]",Point,SARL FELINA,44886929700011,75013,21 AVENUE D ITALIE,Toute l'année,6.41,Point,"[2.3567996474703707, 48.8289332873901]",NaN
3,terrasses-autorisations,3ce3f9815ad4dcb0919edc9ff3796df772cdcd4c,2022-10-12T05:00:08.974Z,2.00,TERRASSE OUVERTE,"[48.852831747101476, 2.3535356286432836]",http://affichette-commerce.paris.fr/2015/Q_16/...,"[2.3535356286432836, 48.852831747101476]",Point,SNC SNC LE GLACIER,39012521900019,75004,4 RUE JEAN DU BELLAY,Toute l'année,4.60,Point,"[2.3535356286432836, 48.852831747101476]",NaN
4,terrasses-autorisations,d52d761995f13b84e9caaf9b9c6c957b05c58c58,2022-10-12T05:00:08.974Z,0.60,TERRASSE OUVERTE,"[48.84215822099757, 2.3246645838035245]",http://affichette-commerce.paris.fr/2015/Q_53/...,"[2.3246645838035245, 48.84215822099757]",Point,SAS MUEL-SA,82250934500010,75014,15 RUE D ODESSA,Toute l'année,6.50,Point,"[2.3246645838035245, 48.84215822099757]",NaN
5,terrasses-autorisations,9b7e01cabb5b1aab6c3245160e15fbed1120f9e2,2022-10-12T05:00:08.974Z,1.00,TERRASSE OUVERTE,"[48.84043086088117, 2.3241029996465685]",http://affichette-commerce.paris.fr/2015/Q_53/...,"[2.3241029996465685, 48.84043086088117]",Point,M OU MME ALAUX CHRISTIAN THIERRY,38459127700031,75014,10 RUE DE LA GAITE,Toute l'année,3.10,Point,"[2.3241029996465685, 48.84043086088117]",CAFE GAITE
6,terrasses-autorisations,875bc8d61b221080e2e2db3eac094673e14f0983,2022-10-12T05:00:08.974Z,1.42,TERRASSE OUVERTE,"[48.864795217581346, 2.3596171561033144]",http://affichette-commerce.paris.fr/2015/Q_09/...,"[2.3596171561033144, 48.864795217581346]",Point,SARL SOCIAL SQUARE,72204637200016,75003,165 RUE DU TEMPLE,Toute l'année,5.30,Point,"[2.3596171561033144, 48.864795217581346]",SOCIAL SQUARE
7,terrasses-autorisations,281053ecef7370e8e95752ce619f2756e3f4f91e,2022-10-12T05:00:08.974Z,0.60,TERRASSE OUVERTE,"[48.88566426612595, 2.3660185294935587]",http://affichette-commerce.paris.fr/2015/Q_73/...,"[2.3660185294935587, 48.88566426612595]",Point,SARL CASABLANCA,44046301600012,75019,26 RUE D AUBERVILLIERS,Toute l'année,5.22,Point,"[2.3660185294935587, 48.88566426612595]",CASABLANCA
8,terrasses-autorisations,8ee8220b169ea7a9dee79935b15dbf7d655a7fe0,2022-10-12T05:00:08.974Z,1.00,TERRASSE OUVERTE,"[48.879015311863185, 2.291055674220488]",http://affichette-commerce.paris.fr/2015/Q_65/...,"[2.291055674220488, 48.879015311863185]",Point,M OU MME PALMIE MAURICE LEOPOLD MARCEL,31078382400025,75017,34 RUE D ARMAILLE,Toute l'année,1.60,Point,"[2.291055674220488, 48.879015311863185]",Civilité : MONSIEUR
9,terrasses-autorisations,89d53cffeacabed62769c7db88061cb751726bcb,2022-10-12T05:00:08.974Z,0.70,TERRASSE OUVERTE,"[48.84886955329743, 2.347263622459078]",http://affichette-commerce.paris.fr/2015/Q_20/...,"[2.347263622459078, 48.84

In [1]:
numbers = [-2, 2, 7, -3]

numbers_count = len(numbers)

numbers.append(5)
numbers.remove(7)

In [2]:
numbers

[-2, 2, -3, 5]